In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold

import pickle
import sys,os
import re
import jieba
jieba.load_userdict("../../MLModel/code/WordCut/userdict.txt")
import string

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.801 seconds.
Prefix dict has been built succesfully.


In [60]:
sys.path.append('../../MLModel/code/TimePattern/')
sys.path.append('../../Lib/SUPPORT')
from  time_pattern import TimePattern
from SUPPORT import balance_category
t = TimePattern('../../MLModel/code/TimePattern/mapping.csv')
kf = KFold(n_splits=10, shuffle=False, random_state=None)

In [63]:
def sub_df(df,sets,target='label'):
    result = pd.DataFrame()
    for each in sets:
        result = pd.concat([result,df[df[target]==each]])
#     print(result[target].value_counts())
    return result


def cut_words(text):
    ##### more -- added by wei
    # this is used to remove time patterns from sentence
    text = re.sub(r' ','',text)
    text = t.remove_time(text)
    #########
    seg_list = jieba.cut(text, cut_all=False)
    return " ".join(seg_list)

def clean(text):
    text = re.sub(f'([{string.punctuation}“”¨«»®´·º ½¾¿¡§£₤‘’，])',' ', text)
    text = text.split(' ')
    text = ' '.join(text)
    return text

def clean_label(label):
    return int(label)

others = pd.read_csv('../../MLModel/data/others/irrelevant_response_training_set.csv')
other_matrix = pd.read_csv('../../MLModel/data/others/strategy_mat.csv')

others = others.rename({'文本':'original_text','类别':'from'},axis=1)
mapping = other_matrix.set_index('category').label.drop_duplicates()
others['original_label'] = others['from'].map(mapping)

others['split_text']=others['original_text'].apply(cut_words)
    
# # cleaning and save
others['split_text'] = others['split_text'].apply(clean)

data = others.copy()
data = data.sample(frac=1,random_state=19)

# K fold
kf = KFold(n_splits=10, shuffle=False, random_state=None)
ss = kf.split(data)
result = pd.DataFrame()

all_preds = np.array([])

for train_index,val_index in ss:
    train_df = data.iloc[train_index]
    train_df = balance_category(train_df,target='original_label')
    val_df = data.iloc[val_index].copy()
    train_data = train_df.split_text.values
    val_data = val_df.split_text.values
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                strip_accents='unicode', 
                                max_features=100000, 
                                analyzer='word',
                                sublinear_tf=True,
                                token_pattern=r'\w{1,}')


    phrase_vectorizer.fit(train_data)
    train_tfidf = phrase_vectorizer.transform(train_data)
    val_tfidf = phrase_vectorizer.transform(val_data)
    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(train_tfidf, train_df.original_label)
    preds = lsvc.predict(val_tfidf)
    val_df['pred_label'] = preds
    val_df = val_df[val_df.original_label != val_df.pred_label]
    result = pd.concat([result,val_df])
    all_preds = np.concatenate([all_preds,preds])





In [64]:
eval_mat(data.original_label.values,all_preds)

,pred_100,pred_101,pred_102,pred_103,pred_104,pred_105,pred_106,pred_107,pred_108,pred_109,recall
actual_100,370.000000,2.000000,14.000000,3.00,29.000000,1.000000,28.000000,1.000000,5.00000,31.00000,0.764463
actual_101,0.000000,180.000000,3.000000,7.00,6.000000,1.000000,6.000000,14.000000,1.00000,38.00000,0.703125
actual_102,10.000000,1.000000,265.000000,3.00,8.000000,4.000000,9.000000,2.000000,3.00000,28.00000,0.795796
actual_103,4.000000,7.000000,2.000000,534.00,11.000000,1.000000,0.000000,7.000000,0.00000,39.00000,0.882645
actual_104,13.000000,7.000000,2.000000,9.00,561.000000,6.000000,11.000000,1.000000,0.00000,57.00000,0.841079
actual_105,3.000000,2.000000,3.000000,3.00,12.000000,174.000000,1.000000,0.000000,1.00000,12.00000,0.824645
actual_106,31.000000,8.000000,3.000000,2.00,9.000000,0.000000,120.000000,0.000000,2.00000,38.00000,0.563380
actual_107,2.000000,11.000000,3.000000,7.00,4.000000,0.000000,0.000000,268.000000,0.00000,41.00000,0.797619
actual_108,10.000000,2.000000,1.000000,2.00,1.000000,5.000000,0.000000,3.000000,174.00000,17.00000,0.809302
actual_109,30.000000,28.000000,18.000000,30.00,55.000000,4.000000,21.000000,38.000000,6.00000,1145.00000,0.832727


In [20]:
sys.path.append('../../Lib/')
from model_matrix import eval_mat
eval_mat(data.original_label.values,all_preds)

,pred_100,pred_101,pred_102,pred_103,pred_104,pred_105,pred_106,pred_107,pred_108,pred_109,recall
actual_100,380.000000,0.000000,20.000000,3.000000,26.000000,1.000000,21.000000,1.000000,9.000000,23.000000,0.785124
actual_101,1.000000,179.000000,3.000000,14.000000,7.000000,1.000000,3.000000,15.000000,3.000000,30.000000,0.699219
actual_102,13.000000,1.000000,274.000000,3.000000,8.000000,4.000000,4.000000,1.000000,5.000000,20.000000,0.822823
actual_103,4.000000,6.000000,2.000000,550.000000,10.000000,1.000000,0.000000,6.000000,0.000000,26.000000,0.909091
actual_104,13.000000,8.000000,3.000000,9.000000,575.000000,6.000000,3.000000,1.000000,0.000000,49.000000,0.862069
actual_105,4.000000,0.000000,3.000000,1.000000,15.000000,177.000000,0.000000,1.000000,1.000000,9.000000,0.838863
actual_106,37.000000,4.000000,7.000000,2.000000,9.000000,0.000000,116.000000,0.000000,4.000000,34.000000,0.544601
actual_107,2.000000,8.000000,5.000000,6.000000,5.000000,1.000000,1.000000,277.000000,0.000000,31.000000,0.824405
actual_108,8.000000,2.000000,1.000000,2.000000,1.000000,7.000000,0.000000,2.000000,178.000000,14.000000,0.827907
actual_109,43.000000,28.000000,19.000000,42.000000,63.000000,9.000000,22.000000,50.000000,16.000000,1083.000000,0.787636


In [67]:
others = pd.read_csv('../../MLModel/data/others/irrelevant_response_training_set.csv')

In [62]:
train_df

,original_text,from,original_label,split_text
2240,稍后给你回复,请求等下打来,104,稍后 给 你 回复
190,这个和你有关系吗？,说出目的,101,这个 和 你 有 关系 吗 ？
10,滚，你tm还骗到老子头上来了。,故意岔开话题,109,滚 你 tm 还 骗 到 老子 头上 来 了 。
1216,我去看看详细合同,确认数额,102,我 去 看看 详细 合同
2977,风大；没听到,请求重复,103,风大 ； 没 听到
629,也许可以,讨价还价,100,也许 可以
172,我跪谢你别给我打电话了哈！,故意岔开话题,109,我 跪谢 你别 给 我 打电话 了 哈 ！
1743,我回家商量下,讨价还价,100,我 回家 商量 下
2147,我问问财务，你一会再联系我,请求等下打来,104,我 问问 财务 你 一会 再 联系 我
1306,利息，滞纳金咋个这么高？你们是咋个算的？,确认数额,102,利息 滞纳金 咋个 这么 高 ？ 你们 是 咋个 算 的 ？


In [32]:
others.to_csv('../../MLModel/data/others/irrelevant_response_training_set.csv',index=False,encoding='utf8')

In [66]:
others

,original_text,from,original_label,split_text
0,我们skype上聊吧,其它通讯方式,105,我们 skype 上聊 吧
1,回去QQ上和你说,其它通讯方式,105,回去 QQ 上 和 你 说
2,你们公司在哪点，我过来,其它通讯方式,105,你们 公司 在 哪点 我 过来
3,微信聊,其它通讯方式,105,微信 聊
4,电话里面说不清，要不你加我 QQ，我们在 QQ 里面聊这个事情，方便些,其它通讯方式,105,电话 里面 说不清 要 不 你 加 我 QQ 我们 在 QQ 里面 聊 这个 事情 ...
5,你发我一个短信，把计算公式和详情发过来一下,其它通讯方式,105,你 发 我 一个 短信 把 计算公式 和 详情 发过来 一下
6,聊一下,其它通讯方式,105,聊 一下
7,联系我助理吧,其它通讯方式,105,联系 我 助理 吧
8,你打我另外个电话,其它通讯方式,105,你 打 我 另外 个 电话
9,你把所有信息通过微信发给我把,其它通讯方式,105,你 把 所有 信息 通过 微信 发给 我 把


In [68]:
others['类别'].value_counts()

故意岔开话题    1395
请求等下打来     669
请求重复       611
讨价还价       484
确认数额       333
回问身份       293
说出目的       264
还款方式       215
模糊确认       213
其它通讯方式     210
Name: 类别, dtype: int64